In [6]:
%pylab inline
import os
import pickle
import json
import pandas as pd
from plotly import express as px
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import scattertext as st

Populating the interactive namespace from numpy and matplotlib


C:\Users\PC\.conda\envs\twitter-vaccine\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning:

pylab import has clobbered these variables: ['text']
`%matplotlib` prevents importing * from pylab and numpy



In [7]:
df = pd.read_pickle('translated_tweets.pkl')
df['translated_text'] = df['translated_text'].apply(lambda x: x['translatedText'])

In [8]:
additional_stopwords = ['na', 'https', 'ha', 'hahaha', 'haha', 'lang', 'ang', 'yan', 'ng', 'sa', 'rin', 'yun', 'yang', 'si', 'ako', 'siya', 'ka', 'po', 
                        'mga', 'yung', 'pa', 'pala', 'na', 'ni', 'sya', 'ba', 'ko', 'nyo', 'man']
additional_stopwords = set(list(ENGLISH_STOP_WORDS) + additional_stopwords)

In [9]:
df['created_at_date'] = pd.to_datetime(df['created_at_date'])
df['YEAR'] = df['created_at_date'].dt.year
df['MONTH'] = df['created_at_date'].dt.month
df['DAY'] = df['created_at_date'].dt.day

In [10]:
for i, tmp in df.sort_values(['YEAR']).groupby(['YEAR']):
    for keyword in ['dengvaxia', 'vacci', 'dengue']:
        if keyword == 'vacci':
            txt = tmp[(tmp['full_text'].str.lower().str.contains('vacci')) | (tmp['full_text'].str.lower().str.contains('bakun'))].copy()
        else:
            txt = tmp[(tmp['full_text'].str.lower().str.contains(keyword))]
        if txt.shape[0] <= 0:
            continue
        tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                    stop_words = additional_stopwords,
                                    ngram_range=(1,4), 
                                    lowercase = True, max_df=0.9)
        print(txt['full_text'].shape)
        dtm_tf = tf_vectorizer.fit_transform(txt['full_text'].str.lower().str.replace('dengue', '').str.replace('vaccine', '').str.replace('dengvaxia', '').str.replace('bakuna', ''))
        lda_tf = LatentDirichletAllocation(n_components=5, random_state=0)
        lda_tf.fit(dtm_tf)
        print(keyword, i)
        p = pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)
        try:
            pyLDAvis.save_html(p, f'plots/ldavis/monthly/{i}-{keyword} - pcoa.html')
        except:
            pass

(2,)
dengvaxia 2015


NameError: name 'pyLDAvis' is not defined

In [5]:

for i, tmp in df.sort_values(['YEAR', 'MONTH']).groupby(['YEAR']):
    print(i)
    keywords = ['dengvax', 'vacci', 'dengue']
    for word in keywords:
        print(word)
        if word == 'vacci':

            tmp2 = tmp[(tmp['translated_text'].str.lower().str.contains('vacci')) | (tmp['translated_text'].str.lower().str.contains('bakuna'))].copy()
        else:
            tmp2 = tmp[tmp['translated_text'].str.lower().str.contains(word)].copy()
        tmp2['keyword'] = word
        tmp3 = tmp[~tmp['translated_text'].str.lower().str.contains(word)].copy()
        tmp3['keyword'] = 'not ' + word
        tmp = tmp2.append(tmp3)
        tmp['translated_text'] = tmp['translated_text'].str.lower()
        tmp['parse'] = tmp['translated_text'].apply(st.whitespace_nlp_with_sentences)

        try:
            unigram_corpus = (st.CorpusFromParsedDocuments(tmp,
                                                       category_col='keyword',
                                                       parsed_col='parse')
                          .build().remove_terms(additional_stopwords, ignore_absences=True).get_stoplisted_unigram_corpus())
        except:
            continue
        topic_model_nmf = st.SentencesForTopicModeling(unigram_corpus).get_topics_from_model(
            Pipeline([
                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                ('nmf', (NMF(n_components=100, alpha=.1, l1_ratio=.5, random_state=0)))
            ]),
            num_terms_per_topic=20
        )

        topic_feature_builder_nmf = st.FeatsFromTopicModel(topic_model_nmf)

        topic_corpus_nmf = st.CorpusFromParsedDocuments(
            tmp,
            category_col='keyword',
            parsed_col='parse',
            feats_from_spacy_doc=topic_feature_builder_nmf
        ).build()
        
        try:
            html = st.produce_scattertext_explorer(
                topic_corpus_nmf,
                category=word,
                width_in_pixels=1000,
                use_non_text_features=True,
                use_full_doc=True,
                pmi_threshold_coefficient=0,
                topic_model_term_lists=topic_feature_builder_nmf.get_top_model_term_lists(),
                topic_model_preview_size=20
            )
            with open(f'plots/scattertext/by_term/translated/{i}_{word} - NMF Topic Model.html', 'w') as f:
                f.write(html)
        except:
            continue

2015
dengvax
vacci
dengue
2016
dengvax
vacci
dengue
2017
dengvax
vacci
dengue
2018
dengvax
vacci
dengue
2019
dengvax
vacci
dengue
